### Рекомендации товаров

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.


В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.
Гарантируется, что среди id купленных товаров все различные.


   * Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
   * Если товар не встречался в обучающей выборке, его популярность равна 0.
   * Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
   * Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.
   
   
   

   
Реализуйте два алгоритма рекомендаций:


* сортировка просмотренных id по популярности (частота появления в просмотренных),
* сортировка просмотренных id по покупаемости (частота появления в покупках).



* Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
* Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k


Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках

In [1]:
import os
import tqdm
import numpy as np
import pandas as pd

from collections import OrderedDict, Counter
from itertools import chain

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'ranking_goods.ipynb',
 'sessions_test.txt',
 'sessions_train.txt',
 'task1.txt',
 'task2.txt',
 'task3.txt',
 'task4.txt',
 '__pycache__']

In [3]:
def preprocess_data(data):
    """
    Выполнить обработку логов.
    Сформировать массив значений Mx2, где M-одна сессия, 0 столбец-просмотры, 1 столбец-заказы.
    """
    data = [x.replace('\n', '').split(';') for x in data]
    data = [[x[0].split(','), x[1].split(',')] for x in data]
    for session_id in range(len(data)):
        data[session_id][0] = [int(x) for x in data[session_id][0]]
        if data[session_id][-1] == ['']:
            data[session_id][-1] = []
        else:
            data[session_id][-1] = [int(x) for x in data[session_id][-1]]
    return np.array(data)

In [4]:
with open('sessions_train.txt') as f:
    train_data = f.readlines()
train_array = preprocess_data(train_data)

with open('sessions_test.txt') as f:
    test_data = f.readlines()
test_array = preprocess_data(test_data)

Расчет частот по просмотрам и заказам по обучающему множеству:

In [5]:
def count_id_frequency(array):
    goods_frequency = Counter()
    for row in array:
        goods_frequency.update(Counter(row))
    return goods_frequency

In [6]:
views_frequency = count_id_frequency(train_array[:, 0])
purchases_frequency = count_id_frequency(train_array[:, 1])

Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.

In [7]:
train_array = np.array([x for x in train_array.tolist() if len(x[1]) > 0])
test_array = np.array([x for x in test_array.tolist() if len(x[1]) > 0])

In [8]:
train_array

array([[list([59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 68, 67]),
        list([67, 60, 63])],
       [list([84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86]),
        list([86])],
       [list([138, 198, 199, 127]), list([199])],
       ...,
       [list([64552, 25931, 2807]), list([25935, 2807])],
       [list([91921, 20251, 5063, 21742, 5063, 20251, 34927]),
        list([91921])],
       [list([32291, 60520, 32291, 38220]), list([32291])]], dtype=object)

In [9]:
test_array

array([[list([63, 68, 69, 70, 66, 61, 59, 61, 66, 68]), list([66, 63])],
       [list([158, 159, 160, 159, 161, 162]), list([162])],
       [list([200, 201, 202, 203, 204]), list([201, 205])],
       ...,
       [list([60538, 44430, 66252, 44430, 60538, 66251]),
        list([66252, 44430])],
       [list([49815, 76363]), list([49815])],
       [list([21841, 17711, 21841, 17711, 21841, 17711, 21841, 17711, 22562, 17711, 21841]),
        list([21841])]], dtype=object)

In [10]:
train_array.shape

(3608, 2)

In [11]:
test_array.shape

(3665, 2)

In [12]:
def clear_duplicates(l):
    """
    Очистить список от дубликатов. При этом сохранить порядок.
    Дубликаты могут быть налюбом расстоянии друг от друга.
    """
    new_l = []
    for x in l:
        if x in new_l:
            pass
        else:
            new_l.append(x)
    return new_l

In [13]:
train_array[0][0]

[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 68, 67]

In [14]:
def recommend(frequency_dict, views, k):

    #unique_views = clear_duplicates(views)
    unique_views = list(OrderedDict.fromkeys(views))
    
    if len(unique_views) < k:
        recomend_list = unique_views
    else:
        df = pd.DataFrame({'id': unique_views,
                           'position': [i for i in range(len(unique_views))],
                           'frequency': [frequency_dict[x] for x in unique_views],
                         } 
                         )
        df.sort_values(['frequency', 'position'], ascending=[False, True], kind='mergesort', inplace=True)
        recomend_list = df['id'].tolist()[:k]
        
    return recomend_list

In [15]:
recommend(views_frequency, train_array[0][0], 5)

[63, 64, 60, 61, 65]

In [16]:
recommend(views_frequency, train_array[0][0], 1)

[63]

In [17]:
def get_metrics(recomends, orders, k):
    if len(orders)==0:
        return {'precision': 0,
                'recall': 0
           }
    else:
        c = len(set(recomends).intersection(set(orders)))
        return {'precision': c/k,
                'recall': c/len(orders)
               }

In [18]:
def evaluate_dataset(dataset, frequency_dict):
    for k in [1, 5]:
        precision_list = []
        recall_list = []
        for i in range(len(dataset)):
            
            views, orders = dataset[i][0], dataset[i][1]
            recomendation = recommend(frequency_dict, views, k)

            metrcis = get_metrics(recomendation, orders, k)
            precision_list.append(metrcis['precision'])
            recall_list.append(metrcis['recall'])

        mean_precision = sum(precision_list)/len(dataset)
        mean_recall = sum(recall_list)/len(dataset)
        print(f'Recall@{k}: {mean_recall}')
        print(f'Precision@{k}: {mean_precision}')

In [19]:
print('Train data. Views frequency')
evaluate_dataset(train_array, views_frequency)
print('\nTrain data. Purchases frequency')
evaluate_dataset(train_array, purchases_frequency)

Train data. Views frequency
Recall@1: 0.4426343165949593
Precision@1: 0.5121951219512195
Recall@5: 0.8246918247126122
Precision@5: 0.21252771618625918

Train data. Purchases frequency
Recall@1: 0.6884494924267653
Precision@1: 0.8037694013303769
Recall@5: 0.9263073024228787
Precision@5: 0.2525498891352649


In [20]:
print('Test data. Views frequency')
evaluate_dataset(test_array, views_frequency)
print('\nTest data. Purchases frequency')
evaluate_dataset(test_array, purchases_frequency)

Test data. Views frequency
Recall@1: 0.41733266203252534
Precision@1: 0.48130968622100956
Recall@5: 0.8000340663538579
Precision@5: 0.2037653478854079

Test data. Purchases frequency
Recall@1: 0.4606201666660294
Precision@1: 0.5276944065484311
Recall@5: 0.8201874337490194
Precision@5: 0.21009549795362173
